In [34]:
import numpy as np
import plotly.express as px
import pandas as pd
import math
import string
from collections import Counter

printable = set(string.printable)


In [2]:
# with open('Jack_London_-_The_Sea_Wolf.txt') as inp:
#     with open('Jack_London_-_The_Sea_Wolf_ascii.txt', 'a') as outp:
#         for line in inp:
#             outp.write(filter(lambda x: x in printable, line) + '\n')


In [3]:
with open('Jack_London_-_The_Sea_Wolf_ascii.txt', 'r') as file:
    text = file.read()


In [4]:
len(text)


577875

Правила:
    1. Заглавная после точки и пробела
    

In [5]:
sum(c.isupper() for c in text)


12142

По правилу - заглавная, в действительности - маленькая

In [6]:
import re
low_char_aft_dot = []
p = re.compile("\. [a-z]")
for m in p.finditer(text):
    low_char_aft_dot.append(m.start()+2)
    # print(m.start()+2, m.group())


In [7]:
low_char_aft_dot


[210266, 371455]

По правилу - маленькая, в действительности - заглавная 

r'(\. [a-z])|( "[a-z])|(\t[a-z])'

In [8]:
upper_inst_lower = []
lower_inst_upper = []


In [9]:
for m in re.compile(r'((?!\. )[A-Z])|'
                    + r'((?! ")[A-Z])|'
                    + r'((?!\t)[A-Z])').finditer(text):
    # low_char_aft_dot.append(m.start()+2)
    upper_inst_lower.append(m.start())
    # print(text[m.start()-2:m.start()+1])#, m.group())# TODO ? !


In [10]:
for m in re.compile(r'(\. [a-z])|( "[a-z])|(\t[a-z])').finditer(text):
    # low_char_aft_dot.append(m.start()+2)
    lower_inst_upper.append(m.end()-1)
    # print(text[m.end()-1], m.group())


In [11]:
# [text[i-3:i+2] for i in upper_inst_lower]


In [12]:
# [text[i-3:i+2] for i in lower_inst_upper]


In [13]:
# r=[]
# for i in range(len(text)):
#     if text[i].isupper() and text[i-2:i]!='. ':
#         r.append(text[i-2:i+1])
# print(r)


In [14]:
exceptions = lower_inst_upper + upper_inst_lower


In [15]:
exceptions_bitmap = [0]*len(text)
for e in exceptions:
    exceptions_bitmap[e] = 1


In [16]:
from collections import Counter, defaultdict

cntr = Counter(exceptions_bitmap)
cntr[1]/cntr[0]


0.02167913396615707

### bitmap as array of numbers

In [17]:
DIGITS_IN_NUBER = 8


In [19]:
"""Add extra zeros to the end of bitmap array so it will have length divisable by DIGITS_IN_NUBER"""
extra_zeros = (DIGITS_IN_NUBER - (len(exceptions_bitmap) %
               DIGITS_IN_NUBER)) % DIGITS_IN_NUBER
bitmap = exceptions_bitmap + [0]*extra_zeros


In [20]:
def gamma_code(number_bits):
    number = int(''.join(map(str, number_bits)), 2)
    number += 1  # кодируем число на 1 больше чтобы могли подавать нули
    i = math.floor(math.log(number, 2))
    # print(f"SPAN number is {i}")
    return [0]*i + [int(c) for c in "{0:b}".format(number)[-(i+1):]]


def gamma_code_array(bits, DIGITS_IN_NUBER):
    res = []
    for iteration in range(len(bits)//DIGITS_IN_NUBER):
        number_bits = bits[iteration *
                           DIGITS_IN_NUBER:(iteration+1)*DIGITS_IN_NUBER]
        res.extend(gamma_code(number_bits))
    return res


In [21]:
len(bitmap) / len(gamma_code_array(bitmap, DIGITS_IN_NUBER=8))


3.6215962147087395

In [22]:
# for i in range(2,17):
#     print(len(bitmap)/ len(gamma_code_array(bitmap, DIGITS_IN_NUBER=i)), i)


In [23]:
gamma_code([0, 0, 0, 0, 0, 0, 1, 0])


[0, 1, 1]

In [24]:
decap = text.lower()


In [25]:
cntr_by_context = {}


In [41]:
for c_beg in range(len(decap)-3):
    context = decap[c_beg:c_beg+3]
    folowing = decap[c_beg+3]
    if context not in cntr_by_context:
        cntr_by_context[context] = defaultdict(int)
    cntr_by_context[context][folowing] += 1


In [27]:
cntr_by_context['ble']


defaultdict(int,
            {' ': 289,
             '.': 37,
             'd': 31,
             'e': 5,
             's': 13,
             'a': 7,
             ',': 29,
             '-': 3,
             ';': 4,
             '!': 1,
             'r': 4,
             'n': 9,
             'w': 5,
             'm': 7,
             '?': 1})

In [28]:
char_cnt_by_context = {k: len(v) for k, v in cntr_by_context.items()}
# char_cnt_by_context


### Необходимый объем памяти для хранения всех моделей

Кодируем 21 бит - контекст длины 3. Далее идет неопределенное количество пар символ-счетчик.
Символ кодируем 7 битами. Счетчик по посхеме с переполнением. Когда пары закончились, нужно просигнализировать о том, что дальше пойдет следующая контекстная модель, а не очередной символ в текущей модели. Запишем 7 нулей.

Кодирование с переполнением по схеме 4+8+8+.... Самое большое значение, которое нужно закодировать - 15658. - Счетчик для "e" в контексте " th"

In [53]:
def overflow_code(x, scheme):
    """scheme == [4, 8, 8] ~ 4 + 8 + 8"""
    exp = scheme[0]
    upper_bound = 2**exp - 2
    if 0 <= x <= upper_bound:
        return "{0:b}".format(x).rjust(exp, '0')
    ones_cnt = exp  # кол-во единиц перед началом числа
    for exp in scheme[1:]:
        prev_upper_bound = upper_bound
        upper_bound = upper_bound + 2**exp - 1
        if x <= upper_bound:
            return '1'*ones_cnt + "{0:b}".format(x-(prev_upper_bound+1)).rjust(exp, '0')
        ones_cnt += exp
    raise ValueError(f"Scheme {scheme} is not enough to code {x}")


def overflow_decode(s, scheme):
    cum_sum_ones_read = 0  # сколько суммарно читается единиц
    lower_bound = 0  # сколько надо вычитать
    for exp_ind in range(len(scheme)):
        potentially_read = cum_sum_ones_read + scheme[exp_ind]
        if s.startswith('1'*potentially_read):
            cum_sum_ones_read = potentially_read
            lower_bound += 2**scheme[exp_ind] - 1
        else:
            return (int(s[cum_sum_ones_read:cum_sum_ones_read+scheme[exp_ind]], 2)
                    + lower_bound)

Всевозможные символы

In [69]:
# sorted(list(set(ord(x) for x in decap)))


In [74]:
# SCH = [8, 16, 16]
SCH = [4, 8, 16]
for i in range(16000):
    encoded = overflow_code(i, SCH)
    decoded = overflow_decode(encoded, SCH)
    assert i == decoded


In [75]:
# Counter(overflow_code(65790, [8, 16, 16]))


In [76]:
ord('a'),chr(97)
# basic latin - 32...127


(97, 'a')

Буквы английского алфавита, стандартные знаки препинания, переводы строк и табы помещаются в первые 128 символов юникода

Хватит 7 бит для того, чтобы закодировать символ.

In [77]:
len(re.findall(' the', text))


7189

In [78]:
cnt_values = []
for context, cntr in cntr_by_context.items():
    for c in cntr:
        cnt_values.append(cntr[c])
        if cntr[c] == 15658:
            print(f"CONTEXT={context} {c}")


fig = px.histogram(cnt_values)
fig.show()
# как часто встречаются значения-счетчики символов. 75% значений счетчиков не превосходит 26
# значит будет выгодно кодировать сетчики по схеме с переполнением


CONTEXT= th e


In [79]:
np.quantile(cnt_values, q=0.75)


26.0

In [80]:
pd.Series(cnt_values).describe()


count    28238.000000
mean        40.928678
std        198.745590
min          2.000000
25%          2.000000
50%          8.000000
75%         26.000000
max      15658.000000
dtype: float64

In [81]:
def code_string(s):
    return ''.join(["{0:b}".format(ord(c)).rjust(7, '0') for c in s])
# len(code_string('the'))


In [82]:
# sorted([(ord(x),x) for x in set(text)])[:20]


In [83]:
# SCH = [8, 16, 16]


def encode_context_models(cntr_by_context):
    result = ''
    for context, cntr in cntr_by_context.items():
        model_encoded = ''
        model_encoded += code_string(context)
        for c, cnt in cntr.items():
            model_encoded += code_string(c)
            model_encoded += overflow_code(cnt, SCH)
            # model_encoded += '0'*7
        result += model_encoded
    return result

result = encode_context_models(cntr_by_context)
print(len(result))


551436


In [84]:
len(result) / (8*1024)

67.31396484375

In [ ]:
class Archiver:
    def __init__(self) -> None:
        pass

    